# THE EFFECT OF PURE METAL'S ATOMIC PROPERTIES AND SURFACE CHARACTERISTICS ON ITS WORK FUNCTION: AN ANALYSIS USING SUPPORT VECTOR REGRESSION MODEL


## Requirement

This prediction  model is coded using Python 3.13.9. All the libraries that required can be installed using this cell below

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Libraries

In [2]:
from mp_api.client import MPRester
from pymatgen.core.periodic_table import Element
from pymatgen.core import Structure, Lattice
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import re

MP_API_KEY = "oaqaUyUUgOtqC6jLsILaljDuuUEvrX89"

## Data Acquisition

### Data from Materials Project

In [3]:
with MPRester(MP_API_KEY) as mpr:
    # Memilih material logam yang dibentuk oleh 1 unsur dan teramati lewat eksperimen
    summary_docs = mpr.materials.summary._search(
        is_metal = True,
        theoretical = False,
        elements = ['Li'],
        nelements = [1, 1],
        fields = ["material_id", "nsites", "volume", "structure", "symmetry"]
    )
    structures = {str(doc.material_id): doc.structure for doc in summary_docs}
    
    # Mengubah data struktur ke sel konvensional
    conventional = {mid: s.to_conventional() for mid, s in structures.items()}
    
    material_id = [doc.material_id for doc in summary_docs]
    
    # Menyeleksi material dari summary_docs yang sifat permukaannya telah dihitung nilainya
    surface_properties_docs = mpr.materials.surface_properties.search(
        material_ids = material_id,
        fields = ["material_id", "pretty_formula", "surfaces"]
    )
    
    # Membuat list dari unsur-unsur yang telah dikumpulkan
    unique_elements = list({doc.pretty_formula for doc in surface_properties_docs})

Retrieving SummaryDoc documents:   0%|          | 0/5 [00:00<?, ?it/s]

Retrieving SurfacePropDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

### Data from pymatgen

In [4]:
def valence_electrons_count(conf:str)->int: # Fungsi untuk menghitung banyaknya elektron valensi dari setiap unsur
    patterns = r'(\d+)([spdfgh])(\d{1,2})'
    subshells = re.findall(patterns,str(conf))
    if not subshells:
        return 0
    
    parsed_subshells = []
    for n, l, e in subshells:
        try:
            parsed_subshells.append((int(n), l, int(e)))
        except ValueError:
            continue
    
    if not parsed_subshells:
        return 0
    
    max_n=max(n for n, l, e in parsed_subshells)
    
    valence = 0
    for n, l, e in parsed_subshells:
        if n == max_n:
            valence+=e
        if l == 'd' and n == max_n - 1:
            valence+=e
        if l == 'f' and n == max_n - 2:
            valence+=e
        if l == 'g' and n == max_n - 3:
            valence+=e
        if l == 'h' and n == max_n - 4:
            valence+=e
    
    return valence

sifat_atomik = []

for simbol in tqdm(unique_elements):
    try:
        unsur = Element(simbol)
        
        config = unsur.electronic_structure
        
        data_unsur = {
            'formula_pretty': simbol,
            'atomic_number': unsur.Z,
            'atomic_radius': unsur.atomic_radius,
            '1st_ionization_energy_eV': unsur.ionization_energy, # Ionisasi pertama
            'electron_affinity_eV': unsur.electron_affinity,
            'electronegativity': unsur.X, # Skala Pauling
            'valence_electrons': valence_electrons_count(config),
            'Youngs_modulus': unsur.youngs_modulus,
            'shear_modulus': unsur.rigidity_modulus,
            'bulk_modulus': unsur.bulk_modulus
        }
        
        # 4. Tambahkan dictionary ke list utama
        sifat_atomik.append(data_unsur)
        
    except Exception as e:
        # Menangani jika ada data yang hilang di pymatgen (jarang terjadi)
        print(f"Gagal mengambil data Pymatgen untuk unsur '{simbol}': {e}")

  0%|          | 0/1 [00:00<?, ?it/s]

### Create Dataframe using pandas

In [5]:
print(summary_docs)

[MPDataDoc<SummaryDoc>(
nsites=3,
volume=61.205660097660434,
symmetry=SymmetryData(crystal_system=<CrystalSystem.trig: 'Trigonal'>, symbol='R-3m', hall=None, number=166, point_group='-3m', symprec=0.1, angle_tolerance=5.0, version='2.5.0'),
material_id=MPID(mp-1018134),
structure=Structure Summary
Lattice
    abc : 3.0786742874458892 3.0786724400000005 7.665407794912054
 angles : 78.41531399066483 78.41531201680868 59.99999672804063
 volume : 61.205660097660434
      A : np.float64(-1.539337269999999) np.float64(-2.66621007) np.float64(0.0)
      B : np.float64(-3.07867244) np.float64(-3.0000000000000004e-08) np.float64(0.0)
      C : np.float64(-1.53933724) np.float64(-0.88873703) np.float64(-7.45647799)
    pbc : True True True
PeriodicSite: Li (-4.618, -2.666, -4.988) [0.777, 0.777, 0.6689]
PeriodicSite: Li (-1.539, -0.8887, -2.469) [0.223, 0.223, 0.3311]
PeriodicSite: Li (0.0, 0.0, 0.0) [-0.0, 0.0, 0.0],
fields_not_requested=['builder_meta', 'elements', 'nelements', 'composition', 

In [6]:
print(conventional)

{'mp-1018134': Structure Summary
Lattice
    abc : 3.0786728257327862 3.0786728257327862 22.36943397000011
 angles : 90.0 90.0 119.99999999999999
 volume : 183.616922173755
      A : np.float64(1.5393364128663931) np.float64(-2.6662088770254146) np.float64(0.0)
      B : np.float64(1.5393364128663931) np.float64(2.6662088770254146) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(22.36943397000011)
    pbc : True True True
PeriodicSite: Li (1.539, -0.8887, 2.469) [0.6667, 0.3333, 0.1104]
PeriodicSite: Li (0.0, 0.0, 4.988) [0.0, 0.0, 0.223]
PeriodicSite: Li (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Li (1.539, 0.8887, 9.925) [0.3333, 0.6667, 0.4437]
PeriodicSite: Li (1.539, -0.8887, 12.44) [0.6667, 0.3333, 0.5563]
PeriodicSite: Li (1.539, -0.8887, 7.456) [0.6667, 0.3333, 0.3333]
PeriodicSite: Li (0.0, 0.0, 17.38) [0.0, 0.0, 0.777]
PeriodicSite: Li (1.539, 0.8887, 19.9) [0.3333, 0.6667, 0.8896]
PeriodicSite: Li (1.539, 0.8887, 14.91) [0.3333, 0.6667, 0.6667], 'mp-1

In [7]:
summary_docs_list = []
for doc in summary_docs:
    summary_docs_list.append({
        "material_id": doc.material_id,
        "symmetry_crystal_system": doc.symmetry.crystal_system if doc.symmetry else None,
        "symmetry_symbol": doc.symmetry.symbol if doc.symmetry else None,
        "nsites": doc.nsites,
        "volume": doc.volume
    })

print(summary_docs_list)

[{'material_id': MPID(mp-1018134), 'symmetry_crystal_system': <CrystalSystem.trig: 'Trigonal'>, 'symmetry_symbol': 'R-3m', 'nsites': 3, 'volume': 61.205660097660434}, {'material_id': MPID(mp-10173), 'symmetry_crystal_system': <CrystalSystem.hex_: 'Hexagonal'>, 'symmetry_symbol': 'P6_3/mmc', 'nsites': 2, 'volume': 40.20915440845307}, {'material_id': MPID(mp-567337), 'symmetry_crystal_system': <CrystalSystem.cubic: 'Cubic'>, 'symmetry_symbol': 'I-43d', 'nsites': 8, 'volume': 161.75350839454015}, {'material_id': MPID(mp-51), 'symmetry_crystal_system': <CrystalSystem.cubic: 'Cubic'>, 'symmetry_symbol': 'Fm-3m', 'nsites': 1, 'volume': 20.347259270998425}, {'material_id': MPID(mp-135), 'symmetry_crystal_system': <CrystalSystem.cubic: 'Cubic'>, 'symmetry_symbol': 'Im-3m', 'nsites': 1, 'volume': 20.34159020777027}]


In [8]:
for doc_entry in summary_docs_list:
    material_id = str(doc_entry['material_id'])
    if material_id in conventional:
        structure = conventional[material_id]
        doc_entry['lattice_a'] = structure.lattice.abc[0]
        doc_entry['lattice_b'] = structure.lattice.abc[1]
        doc_entry['lattice_c'] = structure.lattice.abc[2]
        doc_entry['angle_alpha'] = structure.lattice.angles[0]
        doc_entry['angle_beta'] = structure.lattice.angles[1]
        doc_entry['angle_gamma'] = structure.lattice.angles[2]

print(summary_docs_list)

[{'material_id': MPID(mp-1018134), 'symmetry_crystal_system': <CrystalSystem.trig: 'Trigonal'>, 'symmetry_symbol': 'R-3m', 'nsites': 3, 'volume': 61.205660097660434, 'lattice_a': 3.0786728257327862, 'lattice_b': 3.0786728257327862, 'lattice_c': 22.36943397000011, 'angle_alpha': 90.0, 'angle_beta': 90.0, 'angle_gamma': 119.99999999999999}, {'material_id': MPID(mp-10173), 'symmetry_crystal_system': <CrystalSystem.hex_: 'Hexagonal'>, 'symmetry_symbol': 'P6_3/mmc', 'nsites': 2, 'volume': 40.20915440845307, 'lattice_a': 3.0147964037208994, 'lattice_b': 3.0147964037208994, 'lattice_c': 5.1083222, 'angle_alpha': 90.0, 'angle_beta': 90.0, 'angle_gamma': 120.00000000000001}, {'material_id': MPID(mp-567337), 'symmetry_crystal_system': <CrystalSystem.cubic: 'Cubic'>, 'symmetry_symbol': 'I-43d', 'nsites': 8, 'volume': 161.75350839454015, 'lattice_a': 6.864800199999998, 'lattice_b': 6.864800199999998, 'lattice_c': 6.864800199999998, 'angle_alpha': 90.0, 'angle_beta': 90.0, 'angle_gamma': 90.0}, {'m

In [9]:
summary_docs_df = pd.DataFrame(summary_docs_list)

summary_docs_df

,material_id,symmetry_crystal_system,symmetry_symbol,nsites,volume,lattice_a,lattice_b,lattice_c,angle_alpha,angle_beta,angle_gamma
0,mp-1018134,Trigonal,R-3m,3,61.205660,3.078673,3.078673,22.369434,90.0,90.0,120.0
1,mp-10173,Hexagonal,P6_3/mmc,2,40.209154,3.014796,3.014796,5.108322,90.0,90.0,120.0
2,mp-567337,Cubic,I-43d,8,161.753508,6.864800,6.864800,6.864800,90.0,90.0,90.0
3,mp-51,Cubic,Fm-3m,1,20.347259,4.333665,4.333665,4.333665,90.0,90.0,90.0
4,mp-135,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0


In [10]:
print(surface_properties_docs)

[MPDataDoc<SurfacePropDoc>(
surfaces=[SurfaceEntry(miller_index=[2, 1, 1], surface_energy_EV_PER_ANG2=0.033590684296725674, surface_energy=0.5381820701457396, is_reconstructed=False, structure="# generated using pymatgen\ndata_Li\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   2.97629537\n_cell_length_b   4.86097587\n_cell_length_c   33.62578067\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.06395376\n_cell_angle_gamma   90.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Li\n_chemical_formula_sum   Li12\n_cell_volume   486.48740324\n_cell_formula_units_Z   12\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li1  1  0.991395  0.000000  0.770132  1\n  Li  Li2  1  0.296057  0.500000  0.977166  1\n  Li  Li3  1  0.709337  0.000000  0.9373

In [11]:
# Extract surface properties for all materials
surface_properties_lists = []
for doc in surface_properties_docs:
    for surface in doc.surfaces:
        surface_properties_lists.append({
            'material_id': doc.material_id,
            'pretty_formula': doc.pretty_formula,
            'work_function': surface.work_function,
            'miller_index': surface.miller_index,
            'surface_energy': surface.surface_energy,
            'fermi_energy': surface.efermi
        })

surface_properties_df = pd.DataFrame(surface_properties_lists)

surface_properties_df

,material_id,pretty_formula,work_function,miller_index,surface_energy,fermi_energy
0,mp-135,Li,2.934587,"[2, 1, 1]",0.538182,-1.523606
1,mp-135,Li,2.987348,"[2, 1, 0]",0.505540,-1.543300
2,mp-135,Li,2.552218,"[2, 2, 1]",0.518409,-1.437900
3,mp-135,Li,2.598845,"[1, 1, 1]",0.543597,-1.445500
4,mp-135,Li,2.809598,"[3, 3, 2]",0.523772,-1.471163
5,mp-135,Li,2.737616,"[3, 2, 2]",0.535270,-1.449900
6,mp-135,Li,2.690052,"[3, 1, 0]",0.497097,-1.433800
7,mp-135,Li,2.936978,"[3, 3, 1]",0.520861,-1.530700
8,mp-135,Li,2.907347,"[3, 1, 1]",0.526949,-1.532400
9,mp-135,Li,2.695328,"[3, 2, 1]",0.534355,-1.472600


In [12]:
# Merge the two dataframes
merged_summary_surface_df_filtered = pd.merge(
    surface_properties_df,
    summary_docs_df,
    left_on=['material_id'],
    right_on=['material_id'],
    how='inner'  # Use 'inner' merge to keep only rows that match in both dataframes
)

merged_summary_surface_df_filtered

,material_id,pretty_formula,work_function,miller_index,surface_energy,fermi_energy,symmetry_crystal_system,symmetry_symbol,nsites,volume,lattice_a,lattice_b,lattice_c,angle_alpha,angle_beta,angle_gamma
0,mp-135,Li,2.934587,"[2, 1, 1]",0.538182,-1.523606,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
1,mp-135,Li,2.987348,"[2, 1, 0]",0.505540,-1.543300,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
2,mp-135,Li,2.552218,"[2, 2, 1]",0.518409,-1.437900,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
3,mp-135,Li,2.598845,"[1, 1, 1]",0.543597,-1.445500,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
4,mp-135,Li,2.809598,"[3, 3, 2]",0.523772,-1.471163,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
5,mp-135,Li,2.737616,"[3, 2, 2]",0.535270,-1.449900,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
6,mp-135,Li,2.690052,"[3, 1, 0]",0.497097,-1.433800,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
7,mp-135,Li,2.936978,"[3, 3, 1]",0.520861,-1.530700,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
8,mp-135,Li,2.907347,"[3, 1, 1]",0.526949,-1.532400,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0
9,mp-135,Li,2.695328,"[3, 2, 1]",0.534355,-1.472600,Cubic,Im-3m,1,20.341590,3.439312,3.439312,3.439312,90.0,90.0,90.0


In [13]:
print(sifat_atomik)

[{'formula_pretty': 'Li', 'atomic_number': 3, 'atomic_radius': 1.45, '1st_ionization_energy_eV': 5.391714996, 'electron_affinity_eV': 0.61804922, 'electronegativity': 0.98, 'valence_electrons': 1, 'Youngs_modulus': 4.9, 'shear_modulus': 4.2, 'bulk_modulus': 11.0}]


In [14]:
atomic_properties_df = pd.DataFrame(sifat_atomik)

atomic_properties_df

,formula_pretty,atomic_number,atomic_radius,1st_ionization_energy_eV,electron_affinity_eV,electronegativity,valence_electrons,Youngs_modulus,shear_modulus,bulk_modulus
0,Li,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0


In [15]:
all_merged_df = pd.merge(
    merged_summary_surface_df_filtered,
    atomic_properties_df,
    left_on = ['pretty_formula'],
    right_on = ['formula_pretty'],
    how = 'left'
)

all_merged_df = all_merged_df.drop(columns = ['formula_pretty'])

all_merged_df

,material_id,pretty_formula,work_function,miller_index,surface_energy,fermi_energy,symmetry_crystal_system,symmetry_symbol,nsites,volume,...,angle_gamma,atomic_number,atomic_radius,1st_ionization_energy_eV,electron_affinity_eV,electronegativity,valence_electrons,Youngs_modulus,shear_modulus,bulk_modulus
0,mp-135,Li,2.934587,"[2, 1, 1]",0.538182,-1.523606,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
1,mp-135,Li,2.987348,"[2, 1, 0]",0.505540,-1.543300,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
2,mp-135,Li,2.552218,"[2, 2, 1]",0.518409,-1.437900,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
3,mp-135,Li,2.598845,"[1, 1, 1]",0.543597,-1.445500,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
4,mp-135,Li,2.809598,"[3, 3, 2]",0.523772,-1.471163,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
5,mp-135,Li,2.737616,"[3, 2, 2]",0.535270,-1.449900,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
6,mp-135,Li,2.690052,"[3, 1, 0]",0.497097,-1.433800,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
7,mp-135,Li,2.936978,"[3, 3, 1]",0.520861,-1.530700,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
8,mp-135,Li,2.907347,"[3, 1, 1]",0.526949,-1.532400,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0
9,mp-135,Li,2.695328,"[3, 2, 1]",0.534355,-1.472600,Cubic,Im-3m,1,20.341590,...,90.0,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0


In [16]:
all_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   material_id               33 non-null     object 
 1   pretty_formula            33 non-null     object 
 2   work_function             33 non-null     float64
 3   miller_index              33 non-null     object 
 4   surface_energy            33 non-null     float64
 5   fermi_energy              33 non-null     float64
 6   symmetry_crystal_system   33 non-null     object 
 7   symmetry_symbol           33 non-null     object 
 8   nsites                    33 non-null     int64  
 9   volume                    33 non-null     float64
 10  lattice_a                 33 non-null     float64
 11  lattice_b                 33 non-null     float64
 12  lattice_c                 33 non-null     float64
 13  angle_alpha               33 non-null     float64
 14  angle_beta  

### Feature Enginering

In [17]:
unique_crystal_system = all_merged_df['symmetry_crystal_system'].unique()
print(unique_crystal_system)

[<CrystalSystem.cubic: 'Cubic'> <CrystalSystem.hex_: 'Hexagonal'>]


In [18]:
unique_symbol = all_merged_df['symmetry_symbol'].unique()
print(unique_symbol)

['Im-3m' 'Fm-3m' 'P6_3/mmc']


In [19]:
grouping = all_merged_df.groupby('symmetry_crystal_system', sort=False)['symmetry_symbol'].unique()

print(grouping)

symmetry_crystal_system
Cubic        [Im-3m, Fm-3m]
Hexagonal        [P6_3/mmc]
Name: symmetry_symbol, dtype: object


In [20]:
def classify_structure(row):
    # Membersihkan input
    system = str(row['symmetry_crystal_system']).split(':')[-1].replace("'>", "").strip()
    symbol = str(row['symmetry_symbol'])

    # Menggunakan match-case pada tuple (system, symbol)
    match (system, symbol):
        
        # --- Kategori Kubik ---
        case ('Cubic', s) if s.startswith('F'):
            return 'FCC'
        case ('Cubic', s) if s.startswith('I'):
            return 'BCC'
        case ('Cubic', s) if s.startswith('P'):
            return 'Simple Cubic'

        # --- Kategori Heksagonal ---
        case ('Hexagonal', 'P6_3/mmc'):
            return 'HCP' # Ini space group spesifik untuk HCP
        case ('Hexagonal', 'P6/mmm'):
            return 'Primitive Hexagonal'

        # --- Kategori Tetragonal ---
        case ('Tetragonal', s) if s.startswith('I'):
            return 'Body-Centered Tetragonal (BCT)'
        
        # --- Kategori Orthorhombic ---
        case ('Orthorhombic', s) if s.startswith('C'):
            return 'Base-Centered Orthorhombic'

        # --- Kategori Monoclinic ---
        case ('Monoclinic', s) if s.startswith('C'):
            return 'Base-Centered Monoclinic'
            
        # --- Kategori Trigonal ---
        case ('Trigonal', s) if s.startswith('R'):
            return 'Rhombohedral'

        # --- Default Case ---
        case _:
            return 'Other' # Untuk yang tidak terpetakan

# -------------------------------------------------------------------
# --- Terapkan fungsi ini untuk membuat kolom baru ---
# -------------------------------------------------------------------

# Pastikan Anda menggunakan DataFrame asli, BUKAN hasil groupby
all_merged_df['structure_type'] = all_merged_df.apply(classify_structure, axis=1)

# -------------------------------------------------------------------
# --- Lihat hasilnya ---
# -------------------------------------------------------------------
print("DataFrame dengan kolom 'structure_type' baru:")
print(all_merged_df[['symmetry_symbol', 'structure_type']].head())

print("\n--- Ringkasan Struktur di Data Anda ---")
print(all_merged_df['structure_type'].value_counts())

DataFrame dengan kolom 'structure_type' baru:
  symmetry_symbol structure_type
0           Im-3m            BCC
1           Im-3m            BCC
2           Im-3m            BCC
3           Im-3m            BCC
4           Im-3m            BCC

--- Ringkasan Struktur di Data Anda ---
structure_type
BCC    13
FCC    13
HCP     7
Name: count, dtype: int64


In [21]:
all_merged_df.head()

,material_id,pretty_formula,work_function,miller_index,surface_energy,fermi_energy,symmetry_crystal_system,symmetry_symbol,nsites,volume,...,atomic_number,atomic_radius,1st_ionization_energy_eV,electron_affinity_eV,electronegativity,valence_electrons,Youngs_modulus,shear_modulus,bulk_modulus,structure_type
0,mp-135,Li,2.934587,"[2, 1, 1]",0.538182,-1.523606,Cubic,Im-3m,1,20.34159,...,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0,BCC
1,mp-135,Li,2.987348,"[2, 1, 0]",0.505540,-1.543300,Cubic,Im-3m,1,20.34159,...,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0,BCC
2,mp-135,Li,2.552218,"[2, 2, 1]",0.518409,-1.437900,Cubic,Im-3m,1,20.34159,...,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0,BCC
3,mp-135,Li,2.598845,"[1, 1, 1]",0.543597,-1.445500,Cubic,Im-3m,1,20.34159,...,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0,BCC
4,mp-135,Li,2.809598,"[3, 3, 2]",0.523772,-1.471163,Cubic,Im-3m,1,20.34159,...,3,1.45,5.391715,0.618049,0.98,1,4.9,4.2,11.0,BCC


In [22]:
all_merged_df = all_merged_df.drop(columns=['symmetry_crystal_system'])

all_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   material_id               33 non-null     object 
 1   pretty_formula            33 non-null     object 
 2   work_function             33 non-null     float64
 3   miller_index              33 non-null     object 
 4   surface_energy            33 non-null     float64
 5   fermi_energy              33 non-null     float64
 6   symmetry_symbol           33 non-null     object 
 7   nsites                    33 non-null     int64  
 8   volume                    33 non-null     float64
 9   lattice_a                 33 non-null     float64
 10  lattice_b                 33 non-null     float64
 11  lattice_c                 33 non-null     float64
 12  angle_alpha               33 non-null     float64
 13  angle_beta                33 non-null     float64
 14  angle_gamma 

In [23]:
coordinates = []
species = ["Li"]

structure = Structure(species, coordinates)#some structure

for s in structure:
        coordinates.append(s.coords) #cartesian coordinates
        #coordinates.append(s.frac_coords) #would give fractional coordinates instead
        species.append(s.specie.Z)
        #species.append(s.specie) #would give strings (e.g. "Fe") instead of atomic number

print(coordinates)

TypeError: Structure.__init__() missing 1 required positional argument: 'coords'